In [1]:
import sys

sys.path.append("..")  # Add the parent directory of 'examples' to the Python path

## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [2]:
from dharitri_sdk import Address

address = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")

print("Address (bech32-encoded)", address.to_bech32())
print("Public key (hex-encoded):", address.to_hex())
print("Public key (hex-encoded):", address.get_public_key().hex())

Address (bech32-encoded) drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [3]:
address = Address.new_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1", "drt")

... or from a raw public key:

In [4]:
pubkey = bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = Address(pubkey, "drt")

Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [5]:
from dharitri_sdk import AddressFactory

factory = AddressFactory("drt")

address = factory.create_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")
address = factory.create_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = factory.create_from_public_key(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))

Addresses can be converted from one representation to another as follows:

In [6]:
print(address.to_bech32())
print(address.to_hex())

drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf
0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


Getting the shard of an address:

In [7]:
from dharitri_sdk import AddressComputer

address_computer = AddressComputer(number_of_shards=3)
print("Shard:", address_computer.get_shard_of_address(address))

Shard: 1


Checking whether an address is a smart contract:

In [8]:
address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qwgcss9")

print("Is contract:", address.is_smart_contract())

Is contract: True


### Changing the default hrp

We have a configuration class, called `LibraryConfig`, that only stores (for the moment) the **default hrp** of the addresses. The default value is `drt`. The hrp can be changed when instantiating an address, or it can be changed in the `LibraryConfig` class, and all the addresses created will have the newly set hrp.  

In [9]:
from dharitri_sdk import Address
from dharitri_sdk import LibraryConfig


print(LibraryConfig.default_address_hrp)
address = Address.new_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
print(address.to_bech32())

LibraryConfig.default_address_hrp = "test"
address = Address.new_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
print(address.to_bech32())

# setting back the default value
LibraryConfig.default_address_hrp = "drt"

drt
drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf
test1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ss5hqhtr


## REWA / DCDT transfers

Create an REWA transfer:

In [10]:
from dharitri_sdk import Transaction, TransactionsConverter

transaction = Transaction(
    sender="drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf",
    receiver="drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000
)

transaction_converter = TransactionsConverter()
print(transaction_converter.transaction_to_dictionary(transaction))

{'nonce': 77, 'value': '1000000000000000000', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 50000, 'data': '', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}


In case you are using a **guarded** account you should also populate the `guardian` and `guardian_signature` fields after creating the transaction.

We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an REWA transfer, but this time with a payload (data):

In [11]:
transaction = Transaction(
    sender="drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf",
    receiver="drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000,
    data=b"for the book"
)

print(transaction_converter.transaction_to_dictionary(transaction))

{'nonce': 77, 'value': '1000000000000000000', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 50000, 'data': 'Zm9yIHRoZSBib29r', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}


Alternatively, we can create an REWA transfer using a **transaction factory** (as we will see below, transaction factories are more commonly used). But before that, we have to create a configuration object (for any factory that we might use):

In [12]:
from dharitri_sdk import TransactionsFactoryConfig

config = TransactionsFactoryConfig(chain_id="D")

The **transaction factory** is parametrized at instantiation, and the transaction is obtained by invoking the `create_transaction...` method:

In [13]:
from dharitri_sdk import TransferTransactionsFactory

transfer_factory = TransferTransactionsFactory(config=config)
alice = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")
bob = Address.new_from_bech32("drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c")

# With "data" field
transaction = transfer_factory.create_transaction_for_native_token_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book"
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '1000000000000000000', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 68000, 'data': 'Zm9yIHRoZSBib29r', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: for the book


Create a single DCDT transfer:

In [14]:
from dharitri_sdk import Token, TokenTransfer

token = Token("TEST-8b028f")
transfer = TokenTransfer(token, 10000)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 410000, 'data': 'RENEVFRyYW5zZmVyQDU0NDU1MzU0MmQzODYyMzAzMjM4NjZAMjcxMA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: DCDTTransfer@544553542d386230323866@2710


Create a single NFT transfer:

Keep in mind, since we are sending a NFT, we **should** set the amount to `1`.

In [15]:
token = Token(identifier="TEST-38f249", nonce=1)
transfer = TokenTransfer(token=token, amount=1)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1213500, 'data': 'RENEVE5GVFRyYW5zZmVyQDU0NDU1MzU0MmQzMzM4NjYzMjM0MzlAMDFAMDFAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: DCDTNFTTransfer@544553542d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a single SFT transfer (almost the same as above, the only difference being that for the transfer we set the desired amount, as an integer):

In [16]:
token = Token(identifier="SEMI-9efd0f", nonce=1)
transfer = TokenTransfer(token=token, amount=5)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1213500, 'data': 'RENEVE5GVFRyYW5zZmVyQDUzNDU0ZDQ5MmQzOTY1NjY2NDMwNjZAMDFAMDVAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: DCDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a multiple DCDT / NFT transfer:

In [17]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1484000, 'data': 'TXVsdGlEQ0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAw', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: MultiDCDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000


Additionally, we also have a method that combines the above methods and is able to identify the kind of transfer that we intend to perform based on it's parameters. The method can be used as follows:

For native token tranfers:

In [18]:
from dharitri_sdk import TransferTransactionsFactory, TransactionsFactoryConfig

transfer_factory = TransferTransactionsFactory(config=TransactionsFactoryConfig(chain_id="D"))

alice = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")
bob = Address.new_from_bech32("drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c")

# With "data" field
transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book".encode()
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '1000000000000000000', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 68000, 'data': 'Zm9yIHRoZSBib29r', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: for the book


And for DCDT transfers:

In [19]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1484000, 'data': 'TXVsdGlEQ0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAw', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: MultiDCDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000


Alternatively, the protocol will support (in the very near future) sending both native and dcdt tokens in the same transaction. If a `native_amount` is provided together with `token_transfers`, the native token will also be included in the `MultiDCDTNFTTrasfer` built-in function call.

In [20]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1745500, 'data': 'TXVsdGlEQ0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwM0A1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAwQDUyNDU1NzQxMmQzMDMwMzAzMDMwMzBAQDBkZTBiNmIzYTc2NDAwMDA=', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'relayerSignature': ''}
Transaction data: MultiDCDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@03@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@524557412d303030303030@@0de0b6b3a7640000


### Decoding Transactions

For example, when sending multiple DCDT and NFT tokens, the receiver field of the transaction is the same as the sender field and also the value is set to `0` because all the information is encoded in the `data` field of the transaction.

For decoding the data field we have a so called `TransactionDecoder`. We fetch the transaction from the network and then use the decoder.

In [21]:
from dharitri_sdk import ProxyNetworkProvider, TransactionDecoder

proxy = ProxyNetworkProvider("https://devnet-api.dharitri.org")
transaction = proxy.get_transaction("3e7b39f33f37716186b6ffa8761d066f2139bff65a1075864f612ca05c05c05d")

decoder = TransactionDecoder()
decoded_transaction = decoder.get_transaction_metadata(transaction)

print(decoded_transaction.to_dict())

GenericError: Url = [https://devnet-api.dharitri.org/transaction/3e7b39f33f37716186b6ffa8761d066f2139bff65a1075864f612ca05c05c05d?withResults=true], error = HTTPSConnectionPool(host='devnet-api.dharitri.org', port=443): Max retries exceeded with url: /transaction/3e7b39f33f37716186b6ffa8761d066f2139bff65a1075864f612ca05c05c05d?withResults=true (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x74120c371840>: Failed to resolve 'devnet-api.dharitri.org' ([Errno -2] Name or service not known)"))

## Relayed Transactions

First, we get the newtwork configuration using the network providers.

In [21]:
from dharitri_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.dharitri.org")
network_config = provider.get_network_config()

### Relayed V1

This is still supported, but we advise you to use Relayed V3 instead.

In [23]:
from pathlib import Path

from dharitri_sdk import (Address, RelayedTransactionsFactory, Transaction,
                            TransactionComputer, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../dharitri_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

inner_tx = Transaction(
    chain_id=network_config.chain_id,
    sender="drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c",
    receiver="drt1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssxfnjku",
    gas_limit=60000000,
    nonce=198,
    data=b"add@05"
)
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")

relayed_tx = factory.create_relayed_v1_transaction(
    inner_transaction=inner_tx,
    relayer_address=relayer
)
relayed_tx.nonce = 2627

print(transaction_converter.transaction_to_dictionary(relayed_tx))

{'nonce': 2627, 'value': '0', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 61052000, 'data': 'cmVsYXllZFR4QDdiMjI2ZTZmNmU2MzY1MjIzYTMxMzkzODJjMjI3MzY1NmU2NDY1NzIyMjNhMjI2NzQ1NmU1NzRmNjU1NzZkNmQ0MTMwNjMzMDZhNmI3MTc2NGQzNTQyNDE3MDdhNjE2NDRiNDY1NzRlNTM0ZjY5NDE3NjQzNTc1MTYzNzc2ZDQ3NTA2NzNkMjIyYzIyNzI2NTYzNjU2OTc2NjU3MjIyM2EyMjQxNDE0MTQxNDE0MTQxNDE0MTQxNDE0NjQxNDE1OTQ2NGUyYjRkNGU1NTY5MzUzODY3NjI0MjMzMzUyYjYzNTY2NzczNGM1OTM5NzI2NzY1NjE2NTQ1M2QyMjJjMjI3NjYxNmM3NTY1MjIzYTMwMmMyMjY3NjE3MzUwNzI2OTYzNjUyMjNhMzEzMDMwMzAzMDMwMzAzMDMwMzAyYzIyNjc2MTczNGM2OTZkNjk3NDIyM2EzNjMwMzAzMDMwMzAzMDMwMmMyMjY0NjE3NDYxMjIzYTIyNTk1NzUyNmI1MTQ0NDEzMTIyMmMyMjczNjk2NzZlNjE3NDc1NzI2NTIyM2EyMjQ3NGM0MzM0NGE1OTM1NGMzMDc1NmY0YTRkNzA0ODc1MzQ0ODUwNTc2NzQ3NDE0ODQ3NmY0NzZhN2E0NzYxMzYzNTJiNzU0ODMwMzAyZjc3NGM1MTU5NDU1MTUyMzA2NDVhNGM2MzUxMzQ

### Relayed V2

This is still supported, but we advise you to use Relayed V3 instead.

In [24]:
from pathlib import Path

from dharitri_sdk import (Address, RelayedTransactionsFactory, Transaction,
                            TransactionComputer, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../dharitri_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

# for the relayedV2 transactions, the gasLimit for the inner transaction should be 0
inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender="drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c",
            receiver="drt1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssxfnjku",
            gas_limit=0,
            nonce=15,
            data=b"add@05"
        )
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")

relayed_tx = factory.create_relayed_v2_transaction(
    inner_transaction=inner_tx,
    inner_transaction_gas_limit=60_000_000,
    relayer_address=relayer
)
relayed_tx.nonce = 37

print(transaction_converter.transaction_to_dictionary(relayed_tx))

{'nonce': 37, 'value': '0', 'receiver': 'drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 60381500, 'data': 'cmVsYXllZFR4VjJAMDAwMDAwMDAwMDAwMDAwMDA1MDAwNjA1MzdlMzBkNTIyZTdjODFiMDc3ZTdlNzE1ODJjMmQ4ZjZiODFlNjllMUAwZkA2MTY0NjQ0MDMwMzVANjUwNzJhOTExOGRiMTQ3NDdhNGYxY2Y4NTdhNmU0MzczZTNlOWVhZDZlYTYwMzZmNTExMWJmNjg4MDNjNWNhZGZkYzQ0ZDM3YTIxYjkyMDQ0OWI1ZTgzYzc4Mzk3NTQyNWMzZjQ3NjAyMDE3M2FjMGZjNzE0ODE2NDZkNGE5MDA=', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'innerTransactions': []}


## Contract ABIs

A contract's ABI describes the endpoints, data structure and events that a contract exposes. While contract interactions are possible without the ABI, they are easier to implement when the definitions are available.

### Load the ABI from a file

In [26]:
from dharitri_sdk.abi import Abi, AbiDefinition

abi_definition = AbiDefinition.load(Path("./contracts/adder.abi.json"))
abi = Abi(abi_definition)

Or even simpler:

In [27]:
abi = Abi.load(Path("./contracts/adder.abi.json"))

### Manually construct the ABI

If an ABI file isn't directly available, but you do have knowledge of the contract's endpoints and types, you can manually construct the ABI. Let's see a simple example:

In [28]:
abi_definition = AbiDefinition.from_dict({
    "endpoints": [{
        "name": "add",
        "inputs": [
            {
                "name": "value",
                "type": "BigUint"
            }
        ],
        "outputs": []
    }]
})

An endpoint with both inputs and outputs:

In [29]:
abi_definition = AbiDefinition.from_dict({
    "endpoints": [
        {
            "name": "foo",
            "inputs": [
                { "type": "BigUint" },
                { "type": "u32" },
                { "type": "Address" }
            ],
            "outputs": [
                { "type": "u32" }
            ]
        },
        {
            "name": "bar",
            "inputs": [
                { "type": "counted-variadic<utf-8 string>" },
                { "type": "variadic<u64>" }
            ],
            "outputs": []
        }
    ]
})

## Contract deployments

### Load the bytecode from a file

In [30]:
from pathlib import Path

bytecode = Path("contracts/adder.wasm").read_bytes()

### Perform a contract deployment

First, let's create a `SmartContractTransactionsFactory`:

In [31]:
from dharitri_sdk import SmartContractTransactionsFactory

factory = SmartContractTransactionsFactory(config)

If the contract ABI is available, provide it to the factory:

In [32]:
abi = Abi.load(Path("contracts/adder.abi.json"))
factory = SmartContractTransactionsFactory(config, abi)

Now, prepare the deploy transaction:

In [33]:
from dharitri_sdk.abi import U32Value

# For deploy arguments, use typed value objects if you haven't provided an ABI to the factory:
args = [U32Value(42)]
# Or use simple, plain Python values and objects if you have provided an ABI to the factory:
args = [42]

deploy_transaction = factory.create_transaction_for_deploy(
    sender=alice,
    bytecode=bytecode,
    arguments=args,
    gas_limit=10000000,
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(deploy_transaction))
print("Transaction data:", deploy_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1yvesqqqqqqqqqqqqqqqqqqqqqqqqyvesqqqqqqqqqqqqqqqqqqqq9r2gz0', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'MDA2MTczNmQwMTAwMDAwMDAxMjkwODYwMDAwMDYwMDAwMTdmNjAwMjdmN2YwMTdmNjAwMjdmN2YwMDYwMDE3ZjAwNjAwMzdmN2Y3ZjAxN2Y2MDAzN2Y3ZjdmMDA2MDAxN2YwMTdmMDI5MDAyMGIwMzY1NmU3NjE5NjI2OTY3NDk2ZTc0NDc2NTc0NTU2ZTczNjk2NzZlNjU2NDQxNzI2Nzc1NmQ2NTZlNzQwMDAzMDM2NTZlNzYwZjY3NjU3NDRlNzU2ZDQxNzI2Nzc1NmQ2NTZlNzQ3MzAwMDEwMzY1NmU3NjBiNzM2OTY3NmU2MTZjNDU3MjcyNmY3MjAwMDMwMzY1NmU3NjEyNmQ0Mjc1NjY2NjY1NzI1Mzc0NmY3MjYxNjc2NTRjNmY2MTY0MDAwMjAzNjU2ZTc2MTc2ZDQyNzU2NjY2NjU3MjU0NmY0MjY5Njc0OTZlNzQ1NTZlNzM2OTY3NmU2NTY0MDAwMjAzNjU2ZTc2MTk2ZDQyNzU2NjY2NjU3MjQ2NzI2ZjZkNDI2OTY3NDk2ZTc0NTU2ZTczNjk2NzZlNjU2NDAwMDIwMzY1NmU3NjEzNmQ0Mjc1NjY2NjY1NzI1Mzc0NmY3MjYxNjc2NTUzNzQ2ZjcyNjUwMDAyMDM2NTZlNzYwZjZkNDI3NTY2NjY2NTcyNTM2NTc0NDI3OTc0NjU3MzAwMDUwMzY1N

When creating transactions using `SmartContractTransactionsFactory`, even if the ABI is available and provided,
you can still use _typed value_ objects as arguments for deployments and interactions.

Even further, you can use a mix of _typed value_ objects and plain Python values and objects. For example:
```
args = [U32Value(42), "hello", { "foo": "bar" }, TokenIdentifierValue("TEST-abcdef")];
```

Setting the transaction nonce, signing a transaction and broadcasting it are depicted in a later section.

### Computing the contract address

Even before broadcasting, at the moment you know the sender address and the nonce for your deployment transaction, you can (deterministically) compute the (upcoming) address of the contract:

In [34]:
from dharitri_sdk import AddressComputer

address_computer = AddressComputer()
contract_address = address_computer.compute_contract_address(
    deployer=Address.new_from_bech32(deploy_transaction.sender),
    deployment_nonce=deploy_transaction.nonce
)

print("Contract address:", contract_address.to_bech32())

Contract address: drt1qqqqqqqqqqqqqpgqak8zt22wl2ph4tswtyc39namqx6ysa2sd8ssg6vu30


### Parsing transaction outcome

In the end, you can parse the results using a `SmartContractTransactionsOutcomeParser`. However, since the `parse_deploy` method requires a `TransactionOutcome` object as input, we need to first convert our `TransactionOnNetwork` object to a `TransactionOutcome`, by means of a `TransactionsConverter`.

In [35]:
from dharitri_sdk import (SmartContractTransactionsOutcomeParser,
                            TransactionsConverter)

converter = TransactionsConverter()
parser = SmartContractTransactionsOutcomeParser()

transaction_on_network = proxy.get_transaction("0a7da74038244790b5bd4cd614c26cd5a6be76a6fcfcfb037974cc116b2ee9c6")
transaction_outcome = converter.transaction_on_network_to_outcome(transaction_on_network)
parsed_outcome = parser.parse_deploy(transaction_outcome)

print(parsed_outcome)

SmartContractDeployOutcome(return_code='', return_message='', contracts=[DeployedSmartContract(address=drt1qqqqqqqqqqqqqpgqws44xjx2t056nn79fn29q0rjwfrd3m43396qzdr496, owner_address=drt1testnlersh4z0wsv8kjx39me4rmnvjkwu8dsaea7ukdvvc9z396qe2mapq, code_hash=384b680df7a95ebceca02ffb3e760a2fc288dea1b802685ef15df22ae88ba15b)])


## Contract upgrades

Contract upgrade transactions are similar to deployment transactions (see above), in the sense that they also require a contract bytecode. In this context, though, the contract address is already known.

In [40]:
contract_address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qwgcss9")
bytecode = Path("./contracts/adder.wasm").read_bytes()

upgrade_transaction = factory.create_transaction_for_upgrade(
    sender=alice,
    contract=contract_address,
    bytecode=bytecode,
    gas_limit=10000000,
    arguments=[42],
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(upgrade_transaction))
print("Transaction data:", upgrade_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qwgcss9', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'dXBncmFkZUNvbnRyYWN0QDAwNjE3MzZkMDEwMDAwMDAwMTI5MDg2MDAwMDA2MDAwMDE3ZjYwMDI3ZjdmMDE3ZjYwMDI3ZjdmMDA2MDAxN2YwMDYwMDM3ZjdmN2YwMTdmNjAwMzdmN2Y3ZjAwNjAwMTdmMDE3ZjAyOTAwMjBiMDM2NTZlNzYxOTYyNjk2NzQ5NmU3NDQ3NjU3NDU1NmU3MzY5Njc2ZTY1NjQ0MTcyNjc3NTZkNjU2ZTc0MDAwMzAzNjU2ZTc2MGY2NzY1NzQ0ZTc1NmQ0MTcyNjc3NTZkNjU2ZTc0NzMwMDAxMDM2NTZlNzYwYjczNjk2NzZlNjE2YzQ1NzI3MjZmNzIwMDAzMDM2NTZlNzYxMjZkNDI3NTY2NjY2NTcyNTM3NDZmNzI2MTY3NjU0YzZmNjE2NDAwMDIwMzY1NmU3NjE3NmQ0Mjc1NjY2NjY1NzI1NDZmNDI2OTY3NDk2ZTc0NTU2ZTczNjk2NzZlNjU2NDAwMDIwMzY1NmU3NjE5NmQ0Mjc1NjY2NjY1NzI0NjcyNmY2ZDQyNjk2NzQ5NmU3NDU1NmU3MzY5Njc2ZTY1NjQwMDAyMDM2NTZlNzYxMzZkNDI3NTY2NjY2NTcyNTM3NDZmNzI2MTY3NjU1Mzc0NmY3MjY1MDAwMjAzNjU2ZTc2MGY2ZDQyNzU2NjY2NjU3MjUzNjU3NDQyN

## Contract interactions

The recommended way to create transactions for calling (and, for that matter, deploying and upgrading) smart contracts is through a `SmartContractTransactionsFactory`.

In [41]:
from dharitri_sdk import SmartContractTransactionsFactory

factory = SmartContractTransactionsFactory(config)

If the contract ABI is available, provide it to the factory:

In [42]:
abi = Abi.load(Path("contracts/adder.abi.json"))
factory = SmartContractTransactionsFactory(config, abi)

### Regular interactions

Now, let's prepare a contract transaction, to call the `add` function of our previously deployed smart contract:

In [43]:
contract_address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgqws44xjx2t056nn79fn29q0rjwfrd3m43396qzdr496")

# For arguments, use typed value objects if you haven't provided an ABI to the factory:
args = [U32Value(42)]
# Or use simple, plain Python values and objects if you have provided an ABI to the factory:
args = [42]

transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=args
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qqqqqqqqqqqqqpgqws44xjx2t056nn79fn29q0rjwfrd3m43396qzdr496', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'YWRkQDJh', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'innerTransactions': []}
Transaction data: add@2a


When creating transactions using `SmartContractTransactionsFactory`, even if the ABI is available and provided,
you can still use _typed value_ objects as arguments for deployments and interactions.

Even further, you can use a mix of _typed value_ objects and plain Python values and objects. For example:
```
args = [U32Value(42), "hello", { "foo": "bar" }, TokenIdentifierValue("TEST-abcdef")];
```

Setting the transaction nonce, signing a transaction and broadcasting it are depicted in a later section.

### Transfer & execute

At times, you may want to send some tokens (native REWA or DCDT) along with the contract call.

For transfer & execute with native REWA, prepare your transaction as follows:

In [44]:
transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=[42],
    native_transfer_amount=1000000000000000000
)

Above, we're sending 1 REWA along with the contract call.

For transfer & execute with DCDT tokens, prepare your transaction as follows:

In [45]:
first_token = Token("TEST-38f249", 1)
first_transfer = TokenTransfer(first_token, 1)

second_token = Token("BAR-c80d29")
second_transfer = TokenTransfer(second_token, 10000000000000000000)

transfers = [first_transfer, second_transfer]

transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=[42],
    token_transfers=transfers
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'sender': 'drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAMDAwMDAwMDAwMDAwMDAwMDA1MDA3NDJiNTM0OGNhNWJlOWE5Y2ZjNTRjZDQ1MDNjNzI3MjQ2ZDhlZWIxODk3NEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAwQDYxNjQ2NEAyYQ==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': '', 'relayer': '', 'innerTransactions': []}
Transaction data: MultiDCDTNFTTransfer@00000000000000000500742b5348ca5be9a9cfc54cd4503c727246d8eeb18974@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@616464@2a


### Parsing transaction outcome

Documentation in this section is preliminary and subject to change.

### Decode transaction events

You might be interested into decoding events emitted by a contract. You can do so by using the [`TransactionEventsParser`](#).

Suppose we'd like to decode a `startPerformAction` event emitted by the [multisig](https://github.com/dharitri/mx-contracts-rs/tree/main/contracts/multisig) contract.

Let's fetch an already processed [transaction](https://devnet-explorer.dharitri.org/transactions/05d445cdd145ecb20374844dcc67f0b1e370b9aa28a47492402bc1a150c2bab4), to serve as an exmple, and convert it to a [TransactionOutcome](https://dharitri.github.io/mx-sdk-py/_modules/dharitri_sdk/core/transactions_outcome_parsers/resources.html#TransactionOutcome).

In [46]:
from dharitri_sdk import ApiNetworkProvider, TransactionsConverter

api = ApiNetworkProvider("https://testnet-api.dharitri.org")
converter = TransactionsConverter()

transaction_on_network = api.get_transaction("6f006c99e45525c94629db2442d9ca27ff088ad113a09f0a3a3e24bcc164945a")
transaction_outcome = converter.transaction_on_network_to_outcome(transaction_on_network)

Now, lets find and parse the event we are interested in.

In [47]:
from dharitri_sdk import TransactionEventsParser, find_events_by_first_topic
from dharitri_sdk.abi import Abi

abi = Abi.load(Path("./contracts/multisig-full.abi.json"))
events_parser = TransactionEventsParser(abi)

[event] = find_events_by_first_topic(transaction_outcome, "startPerformAction")
parsed_event = events_parser.parse_event(event)

print(parsed_event)

namespace(data=namespace(action_id=1, group_id=0, action_data=namespace(0=b'\x019G.\xffh\x86w\x1a\x98/0\x83\xda]B\x1f$\xc2\x91\x81\xe68\x88"\x8d\xc8\x1c\xa6\ri\xe1', __discriminant__=1), signers=[b'<5\xd7\xea\xcb\x1a\x99\xa4\xa0\xcc\xaaS~\xf0\xcc\n\xe1\t\xf4\x1fl+@"\x03\xe63\xbf\xbb\x07\x1f\\']))


## Contract queries

In order to perform Smart Contract queries, we recommend the use of `SmartContractQueriesController`.

You will notice that the `SmartContractQueriesController` requires a `QueryRunner` object at initialization. A `NetworkProvider`, slighly adapted, is used to satisfy this requirement (more details about **network providers** can be found in a later section).

In [48]:
from dharitri_sdk import (ProxyNetworkProvider, QueryRunnerAdapter,
                            SmartContractQueriesController)

contract = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgqqy34h7he2ya6qcagqre7ur7cc65vt0mxrc8qwq64qt")
query_runner = QueryRunnerAdapter(ProxyNetworkProvider("https://devnet-api.dharitri.org"))

query_controller = SmartContractQueriesController(query_runner)


If the contract ABI is available, provide it to the controller:

In [49]:
abi = Abi.load(Path("contracts/adder.abi.json"))
query_controller = SmartContractQueriesController(query_runner, abi)

Query the contract as follows:

In [50]:
data_parts = query_controller.query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

print("Return data (parsed):", data_parts)

Return data (parsed): [5]


For finer control, first create a contract query, then run it and parse the outcome at a later time:

In [51]:
query = query_controller.create_query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

response = query_controller.run_query(query)
data_parts = query_controller.parse_query_response(response)

print("Return code:", response.return_code)
print("Return data (raw):", response.return_data_parts)
print("Return data (parsed):", data_parts)

Return code: ok
Return data (raw): [b'\x05']
Return data (parsed): [5]


## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [43]:
from dharitri_sdk import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

['rich', 'solution', 'blue', 'ice', 'pass', 'clinic', 'beyond', 'chicken', 'people', 'field', 'release', 'planet', 'arrive', 'tornado', 'trip', 'math', 'circle', 'valley', 'describe', 'second', 'cool', 'range', 'devote', 'case']


The mnemonic can be saved to a keystore file:

In [44]:
from pathlib import Path
from dharitri_sdk import UserWallet

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_mnemonic(mnemonic.get_text(), "password")
wallet.save(path / "walletWithMnemonic.json")

Given a mnemonic, one can derive keypairs:

In [45]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key:", secret_key.hex())
print("Public key:", public_key.hex())

Secret key: 2892696376fef33c5f2c76754e2f701073d524588c7a9cd6681bd6d32bea6650
Public key: f481465e5a676ddc0d73157470e61703abf779496fbc90dce2e97a382bc2871c


A keypair can be saved as a JSON wallet:

In [46]:
path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_secret_key(secret_key, "password")
wallet.save(path / "wallet.json", address_hrp="drt")

... or as a PEM wallet (usually not recommended):

In [47]:
from dharitri_sdk import Address, UserPEM

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

label = Address(public_key.buffer, "drt").to_bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(path / "wallet.pem")

## Loading wallets

This is not a very common use-case - you might refer to [signing objects](#signing-objects) instead.

Load a keystore that holds an **encrypted mnemonic** (and perform wallet derivation at the same time):

In [48]:
from dharitri_sdk import UserWallet

secret_key = UserWallet.load_secret_key(Path("../dharitri_sdk/testutils/testwallets/withDummyMnemonic.json"), "password", address_index=0)
address = secret_key.generate_public_key().to_address("drt")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf


Load a keystore that holds an **encrypted secret** key:

In [49]:
secret_key = UserWallet.load_secret_key(Path("../dharitri_sdk/testutils/testwallets/alice.json"), "password")
address = secret_key.generate_public_key().to_address("drt")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf


Load the secret key from a PEM file:

In [50]:
from dharitri_sdk import UserPEM

pem = UserPEM.from_file(Path("../dharitri_sdk/testutils/testwallets/alice.pem"))

print("Secret key:", pem.secret_key.hex())
print("Public key:", pem.public_key.hex())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key: 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [51]:
from dharitri_sdk import UserSigner

signer = UserSigner.from_wallet(Path("../dharitri_sdk/testutils/testwallets/alice.json"), "password")

Creating a `UserSigner` from a PEM file:

In [52]:
signer = UserSigner.from_pem_file(Path("../dharitri_sdk/testutils/testwallets/alice.pem"))

Signing a transaction:

In [53]:
from dharitri_sdk import Transaction, TransactionComputer

tx = Transaction(
    nonce=90,
    sender="drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf",
    receiver="drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c",
    value=1000000000000000000,
    gas_limit=50000,
    chain_id="D"
)

transaction_computer = TransactionComputer()

tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
print("Signature:", tx.signature.hex())

Signature: ee6d665981c00258858923bbbe51e1cfe29911716190a3eedc24e2205100bb05faf3d8133fc973b00409aa18708b41488702d635efefe6a24b4a2b7f66267a00


Signing an arbitrary message:

In [54]:
from dharitri_sdk import Message, MessageComputer

signer_address = signer.get_pubkey().to_address(hrp="drt")
message = Message(b"hello")
message_computer = MessageComputer()

message.signature = signer.sign(message_computer.compute_bytes_for_signing(message))

print("Signature:", message.signature.hex())

Signature: 561bc58f1dc6b10de208b2d2c22c9a474ea5e8cabb59c3d3ce06bbda21cc46454aa71a85d5a60442bd7784effa2e062fcb8fb421c521f898abf7f5ec165e5d0f


## Verifying signatures

Creating a `UserVerifier`:

In [55]:
from dharitri_sdk import Address, UserVerifier

alice = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")
bob = Address.new_from_bech32("drt1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqlqde3c")

alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)

Verifying a signature:

In [56]:
from dharitri_sdk import MessageComputer, TransactionComputer

transaction_computer = TransactionComputer()
message_computer = MessageComputer()

print(f"Is signature of Alice?", alice_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Alice?", alice_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))
print(f"Is signature of Bob?", bob_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Bob?", bob_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))

Is signature of Alice? True
Is signature of Alice? True
Is signature of Bob? False
Is signature of Bob? False


## Creating network providers

It's recommended to use the `dharitri_sdk_network_providers` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [57]:
from dharitri_sdk import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.dharitri.org")

Creating a Proxy provider:

In [58]:
from dharitri_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.dharitri.org")

## Fetching network parameters

In order to fetch network parameters, do as follows:

In [59]:
config = provider.get_network_config()

print("Chain ID:", config.chain_id)
print("Min gas price:", config.min_gas_price)

Chain ID: D
Min gas price: 1000000000


## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [60]:
account_on_network = provider.get_account(alice)

print("Nonce:", account_on_network.nonce)
print("Balance:", account_on_network.balance)

Nonce: 1853
Balance: 3364970609086014407


When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [61]:
from dharitri_sdk import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign & broadcast the transaction(s).

For further reference, please see [nonce management](https://docs.dharitri.org/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction:

In [62]:
alice = Address.new_from_bech32("drt1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssey5egf")

tx = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)

Transaction hash: 999da2b84011ffd6aedb966c8262804bb5280064a77f543f841e4eae96af8013


Broadcast multiple transactions:

In [63]:
tx_1 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_1))
tx_2.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_2))
tx_3.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_3))

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)

Transactions hashes: (3, {'0': '999da2b84011ffd6aedb966c8262804bb5280064a77f543f841e4eae96af8013', '1': '0cf69d20c599334047fb1ec9e392086a5b00a42b688dbd9728d7e979a2555622', '2': 'b2e2c5fbb4fbf9be8ec539af4cb719c5b478067eb6772fa0a47a031ef4150b5b'})


Now let's fetch a previously-broadcasted transaction:

In [64]:
tx_on_network = provider.get_transaction("9270a6879b682a7b310c659f58b641ccdd5f083e5633669817130269e5b0939b", with_process_status=True)
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)

Status: success
Is completed: True
